# Segment

## 规则分词

### 1. Maximum Match Method

In [19]:
dictionary = {'研究', '研究生', '生命', '命', '的', '起源'}
def tokenize_mm(document):  
    max_length = max([len(word) for word in dictionary])
    start = 0
    words = list()
    while start < len(document):
        length = max_length
        while length >= 1:
            if s[start: start + length] in dictionary or length == 1:
                words.append(s[start: start + length])
                start += length
                break
            else:
                length -= 1
    return words
    
    
s = '研究生命的起源'
print(tokenize_mm(s))

['研究生', '命', '的', '起源']


### 2. Reverse Maximum Match Method

In [20]:
dictionary = {'研究', '研究生', '生命', '命', '的', '起源'}     
def tokenize_rmm(document):
    max_length = max([len(word) for word in dictionary])
    end = len(document)
    words = list()
    while end > 0:
        length = max_length
        while length >= 1:
            if s[end - length: end] in dictionary or length == 1:
                words.append(s[end - length: end])
                end -= length
                break
            else:
                length -= 1
    words = list(reversed(words))
    return words
    
    
s = '研究生命的起源'
print(tokenize_rmm(s))

['研究', '生命', '的', '起源']


### 3. Bi-direction Matching Method

In [21]:
def tokenize_bd(document):
    words_mm = tokenize_mm(document)
    words_rmm = tokenize_rmm(document)
    if len(words_mm) != len(words_rmm):
        return words_mm if len(words_mm) < len(words_rmm) else words_rmm
    elif words_mm == words_rmm:
        return words_mm
    else:
        single_mm = sum([1 for word in words_mm if len(word) == 1])
        single_rmm = sum([1 for word in words_rmm if len(word) == 1])
        return words_mm if single_mm < single_rmm else words_rmm

s = '研究生命的起源'
print(tokenize_bd(s))

['研究', '生命', '的', '起源']


## 统计分词

### 1. Hidden Markov Model (HMM)

1. 给定句子$\lambda$后某组状态$o$的概率可以表示为$ \text{P}(o | \lambda) $, 根据贝叶斯定理,得到$ P(o|\lambda)  = \dfrac{P(\lambda | o) P(o)}{P(\lambda)} $. 由于给定的句子固定,所以如果只是找最大的概率的话,$ P(\lambda) $为常数可以不考虑,只需要找到$ P(\lambda | o) P(o) $最大的$o$就可以了
        

2. 计算$ P(\lambda | o)P(o) $的意思就是说给定状态的组合$o$,计算得到这个句子$\lambda$的概率.假设这个状态组合是$o_1 o_2 ... o_T$,而句子是$\lambda_1 \lambda_2 ... \lambda_T$        

3. 对于$ P(\lambda | o) $,我们采取最简单的每个状态独立的假设,所以$ P(\lambda | o) = P(\lambda_1 | o_1) P(\lambda_2 | o_2) ... P(\lambda_T | o_T) $; 对于$ P(o) $,我们采取2-gram,所以$ P(o) = P(o_1) P(o_2 | o_1) P(o_3 | o_2) ... P(o_T | o_{T-1}) $

4. 综上所述, $ P(\lambda|o) = P(\lambda_1 | o_1) P(o_1) P(\lambda_2 | o_2) P(o_2 | o_1) P(\lambda_3 | o_3) P(o_3 | o_2) ... P(\lambda_T | o_T) P(o_T | o_{T-1}) $

5. 根据Viterbi算法,我们可以通过这种方法找到最大的概率. 假设timestep=t, 那么$ P( o_1 o_2 ... o_t |\lambda_1 \lambda_2 ... \lambda_t) = max( P( o_1 o_2 ... o_{t-1} |\lambda_1 \lambda_2 ... \lambda_{t-1} ) P(o_t | o_{t-1}), o_{t-1} \in ['B', 'M', 'E', 'S'] ) P(\lambda_t | o_t)  $, 而初始情况是:
$ P(o_1|\lambda_1) = P(o_1) P(\lambda_1 | o_1) $


In [12]:
import os
import pickle
class HMM(object):
    def __init__(self, model_filename, corpus_filename):
        self.states = ['B', 'M', 'E', 'S']
        self.model_filename = model_filename
        if os.path.exists(self.model_filename):
            with open(model_filename, 'rb') as f:
                self.init_prob = pickle.load(f)
                self.trans_prob = pickle.load(f)
                self.emit_prob = pickle.load(f)
        else:
            self.init_prob = dict()
            self.trans_prob = dict()
            self.emit_prob = dict()
            for state in self.states:
                self.init_prob[state] = 0.0
                self.trans_prob[state] = {state1: 0.0 for state1 in self.states}
                self.emit_prob[state] = dict()
            _train(corpus_filename)
    
    def _train(self, corpus_filename):
        def create_states(word):
            states = list()
            if len(word) == 1:
                states.append('S')
            elif len(word) > 1:
                states = ['B'] + ['M'] * (len(word) - 2) + ['E']
            return states
            
        num_non_empty_lines = 0
        with open(corpus_filename, encoding='utf8') as f:
            lines = f.readlines()
            for line in lines:
                line = line.strip()
                if not line:
                    continue

                num_non_empty_lines += 1
                
                words = line.split()
                line_states = list()
                chs = list()
                for word in words:
                    line_states.extend(create_states(word))
                    for ch in word:
                        chs.extend(ch)
                
                # update init_prob, trans_prob, emit_prob
                self.init_prob[line_states[0]] += 1
                for index in range(len(line_states)):
                    if index != 0:
                        self.trans_prob[line_states[index - 1]][line_states[index]] += 1
                    self.emit_prob[line_states[index]][chs[index]] = self.emit_prob[line_states[index]].get(chs[index], 0) + 1.0
        
        # normalize
        self.init_prob = {state: count / num_non_empty_lines for state, count in self.init_prob.items()}
        self.trans_prob = {state: {dest_state: count / sum(value.values()) for dest_state, count in value.items()} for state, value in self.trans_prob.items()}
        self.emit_prob = {state: {word: (prob + 1) / sum(word_prob.values()) for word, prob in word_prob.items()} for state, word_prob in self.emit_prob.items()}
        
        # dump
        with open(self.model_filename, 'wb') as f:
            pickle.dump(self.init_prob, f)
            pickle.dump(self.trans_prob, f)
            pickle.dump(self.emit_prob, f)
    
    def viterbi(self, document):
        states = {} # states['E']的'E'表示这条路径的终点是'E',并且是终点是'E'的所有路径中的最优路径
        path = [{}]
        # init path
        for state in self.states:
            path[0][state] = self.init_prob[state] * self.emit_prob[state].get(document[0], 0)
            states[state] = [state]
        # go
        for i in range(1, len(document)):
            unknown = True
            for state in self.states:
                unknown = unknown and document[i] not in self.emit_prob[state].keys()
            
            tmp_states = {}
            
            path.append({})
            for state in self.states:
                emit_p = self.emit_prob[state].get(document[i], 0) if not unknown else 1.0
                path[i][state], previous_state = max([(path[i - 1][previous_state] * self.trans_prob[previous_state][state] * emit_p, previous_state) for previous_state in self.states])
                
                tmp_states[state] = states[previous_state] + [state]
            states = tmp_states
                
        # select the best path
        if self.emit_prob['M'].get(document[-1], 0) > self.emit_prob['S'].get(document[-1], 0):
            (prob, state) = max([(path[len(document) - 1][state], state) for state in ['E', 'M']])
        else:
            (prob, state) = max([(path[len(document) - 1][state], state) for state in self.states])
        
            
        return prob, states[state]
    
    def tokenize(self, document):
        prob, states = self.viterbi(document)
        words = list()
        index = 0
        while index < len(states):
        #for index, state in enumerate(states):
            if states[index] == 'B':
                end = states[index:].index('E') + index + 1
                words.append(document[index: end])
                index = end
            elif states[index] == 'S':
                words.append(document[index])
                index += 1
        return words
        
model_filename = 'data/hmm.pkl'
corpus_filename = 'data/trainCorpus.txt_utf8'
hmm = HMM(model_filename, corpus_filename)

#print(hmm.emit_prob)
print(hmm.emit_prob['E']['案'])
print(hmm.emit_prob['B']['方'])

document = '这是一个非常棒的方案！'
words = hmm.tokenize(document)
print(words)

0.0010421077785925357
0.004205160552198598
['这是', '一个', '非常', '棒', '的', '方案', '！']
